# Graph Embedding Techniques in Python
This notebook introduces different graph embedding methods and demonstrates their implementation.

# Node Embedding Methods
Node embedding techniques aim to represent each node in a graph as a low-dimensional vector while preserving the structural and semantic relationships between nodes.

## 1. Encoder-Decoder for Node Embedding
Autoencoders can be used to learn node embeddings by reconstructing the adjacency matrix or other graph structures.

### **Graph Autoencoder (GAE) Objective Function:**
$$ \mathcal{L} = \sum_{(i,j) \in E} || A_{ij} - \hat{A}_{ij} ||^2 $$
- $ A $ is the adjacency matrix.
- $ \hat{A} $ is the reconstructed adjacency matrix.
- The loss function minimizes the reconstruction error, ensuring embeddings capture the graph structure.

## **Example Run for Graph Autoencoder (GAE)**
Now, let's apply our **Graph Autoencoder (GAE)** to learn node embeddings and reconstruct the adjacency matrix.

In [4]:
import torch
import networkx as nx
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GCNConv

# Define Graph Autoencoder (GAE)
class GraphAutoencoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GraphAutoencoder, self).__init__()
        self.encoder = GCNConv(in_channels, hidden_channels)

    def encode(self, x, edge_index):
        return self.encoder(x, edge_index).relu()

    def decode(self, z, edge_index):
        return torch.sigmoid(torch.sum(z[edge_index[0]] * z[edge_index[1]], dim=1))

# Generate a sample graph
G = nx.karate_club_graph()
data = from_networkx(G)
data.x = torch.eye(data.num_nodes)  # Use identity matrix as node features

# Instantiate and train GAE
model = GraphAutoencoder(in_channels=data.x.size(1), hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)
    loss = F.mse_loss(model.decode(z, data.edge_index), torch.ones(data.edge_index.size(1)))
    loss.backward()
    optimizer.step()
    return loss.item()

# Train the model
for epoch in range(200):
    loss = train()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

# Generate node embeddings
model.eval()
z = model.encode(data.x, data.edge_index)
print('Node Embeddings Shape:', z.shape)

Epoch 0, Loss: 0.2371
Epoch 20, Loss: 0.0055
Epoch 40, Loss: 0.0001
Epoch 60, Loss: 0.0000
Epoch 80, Loss: 0.0000
Epoch 100, Loss: 0.0000
Epoch 120, Loss: 0.0000
Epoch 140, Loss: 0.0000
Epoch 160, Loss: 0.0000
Epoch 180, Loss: 0.0000
Node Embeddings Shape: torch.Size([34, 16])


### **Explanation of the Example Run**
- We generate a **Karate Club Graph** (a well-known social network dataset).
- We use an **identity matrix** as initial node features.
- The **Graph Autoencoder (GAE)** learns embeddings by reconstructing the adjacency matrix.
- The model is trained for **200 epochs** using Mean Squared Error (MSE) loss.
- The **node embeddings** are extracted from the encoder after training.

## 2. DeepWalk for Node Embedding
DeepWalk generates random walks from each node and uses **Skip-gram (Word2Vec)** to learn node embeddings.

### **DeepWalk Learning Objective:**
$$ \max \sum_{v \in V} \sum_{u \in N(v)} \log P(u | v) $$
- $ V $ is the set of nodes.
- $ N(v) $ represents the context nodes obtained via random walks.
- The probability $ P(u | v) $ is modeled using Skip-gram with negative sampling.

In [2]:
!pip install node2vec networkx
import networkx as nx
from node2vec import Node2Vec

# Create a sample graph
G = nx.karate_club_graph()

# Train DeepWalk (using Node2Vec with p=1, q=1)
node2vec = Node2Vec(G, dimensions=128, walk_length=40, num_walks=10, workers=4, p=1, q=1)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Get embedding of a node
print(model.wv['0'])  # Example node embedding

Defaulting to user installation because normal site-packages is not writeable


Computing transition probabilities:   0%|          | 0/34 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 2/2 [00:00<00:00, 249.81it/s]


[ 0.03371138  0.00471057  0.13197999  0.00064104 -0.09922037 -0.18864591
  0.14211093 -0.04652401 -0.14178273  0.02769512  0.27235723 -0.05567688
 -0.08297646 -0.07268335  0.09086467 -0.0256397  -0.04040011  0.05774299
 -0.10838906  0.06713866  0.22494091  0.14894608  0.05475391  0.15392075
 -0.12386379  0.11761998 -0.14566028  0.16910063  0.06333711 -0.06941671
 -0.32792237 -0.03276331  0.05102498 -0.15237664 -0.06230216 -0.0258521
  0.22699015  0.02365667  0.21208745  0.00715821 -0.02499608  0.16354631
 -0.07037735 -0.07002616 -0.01117198  0.15262796 -0.01470388 -0.12951931
  0.02840152  0.03755466  0.15808448  0.02348655  0.16327459  0.04995153
  0.05497856  0.10872374  0.14379376 -0.09098054 -0.14609477  0.04737015
  0.12490214 -0.15646265  0.00324802  0.05044365  0.05020976 -0.1353709
  0.12380543  0.06324374  0.0078472  -0.08213446  0.12353051 -0.01871051
 -0.19025104 -0.04672351 -0.02414738 -0.03031775 -0.05788708 -0.06149279
  0.05448417  0.22005065  0.11390583 -0.12496471  0.0

## 3. Node2Vec for Node Embedding
Node2Vec extends DeepWalk by introducing hyperparameters **p** and **q** to control random walk behavior.

### **Node2Vec Transition Probability:**
$$ P(v_i | v) = \frac{w_{v v_i}}{Z} $$
where:
- $ w_{v v_i} $ is the weight of the edge between nodes $ v $ and $ v_i $.
- $ Z $ is a normalization factor.

The hyperparameters **p** and **q** define the walk strategy:
- **p > 1**: Biased towards depth-first search (DFS) (captures structural similarity).
- **q > 1**: Biased towards breadth-first search (BFS) (captures neighborhood similarity).

In [3]:
# Train Node2Vec with custom parameters
node2vec = Node2Vec(G, dimensions=128, walk_length=40, num_walks=10, workers=4, p=0.5, q=2)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Get embedding of a node
print(model.wv['1'])  # Example node embedding

Computing transition probabilities:   0%|          | 0/34 [00:00<?, ?it/s]

[-0.1202903  -0.12322044  0.01186979 -0.15394455 -0.02900799 -0.1470373
 -0.11592245  0.04669889  0.01383069  0.06194183  0.34580207  0.102961
 -0.0597943  -0.10464931 -0.03758498 -0.12749052 -0.11810691  0.07493912
 -0.191813   -0.04454722  0.03773009 -0.08224007  0.09282345  0.09029263
 -0.08777487  0.13681558 -0.10452212  0.03598142  0.12324533 -0.0466882
 -0.3003342  -0.02999849 -0.06051234 -0.05260294  0.09246498  0.15638356
  0.06814784 -0.09943837  0.20739974  0.05369433  0.04821846  0.14943299
 -0.07356099 -0.1598666  -0.07103743 -0.03823644 -0.02144143 -0.16925715
  0.16066977 -0.10602353  0.26623416 -0.068403    0.17112988  0.11798132
  0.00299158  0.12068678  0.05099092 -0.05239482 -0.06737599  0.1251624
  0.12514053 -0.11338504 -0.00627189  0.13145149  0.02280507  0.07204564
  0.1771723   0.20161648 -0.06503736 -0.16277233  0.07037327 -0.28126395
 -0.09674021 -0.16563804  0.01831471 -0.1867492  -0.05197934 -0.14526072
  0.14737658  0.24008416  0.07780101 -0.2564623   0.1500

Generating walks (CPU: 2): 100%|██████████| 3/3 [00:00<00:00, 138.25it/s]


### **Summary of Node Embedding Methods**
- **Graph Autoencoder (GAE)**: Learns node embeddings by reconstructing the adjacency matrix.
- **DeepWalk**: Uses random walks + Word2Vec for unsupervised learning.
- **Node2Vec**: Extends DeepWalk with flexible random walk strategies.

# Graph Embedding - Whole Graph Representation
In this section, we will explore techniques for embedding entire graphs rather than individual nodes.

## 4. Graph Convolutional Networks (GCN) + Global Pooling
Graph Convolutional Networks (GCN) learn node embeddings using graph convolutions. To embed an **entire graph**, we aggregate node embeddings using **global pooling**.

### **GCN Update Rule:**
$$ H^{(l+1)} = \sigma( \tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} H^{(l)} W^{(l)} ) $$
- $ \tilde{A} $ is the adjacency matrix with self-loops.
- $ \tilde{D} $ is the degree matrix.
- $ W^{(l)} $ is the learnable weight matrix.
- $ \sigma $ is an activation function (e.g., ReLU).

In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

# Load a dataset of graphs
dataset = TUDataset(root='data', name='MUTAG')
loader = DataLoader(dataset, batch_size=32, shuffle=True)

class GraphEmbeddingGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphEmbeddingGNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return global_mean_pool(x, batch)  # Pooling for graph embedding

# Example training loop
model = GraphEmbeddingGNN(dataset.num_node_features, 64, 32)
for batch in loader:
    embeddings = model(batch)
    print(embeddings.shape)  # (batch_size, embedding_dim)

torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([28, 32])


Processing...
Done!


## 5. Graph2Vec
Graph2Vec is inspired by **Doc2Vec** and learns graph embeddings using the Weisfeiler-Lehman (WL) algorithm to generate graph substructure representations.

### **Graph2Vec Objective Function:**
$$ \max \sum_{G \in D} \sum_{h \in H(G)} \log P(h | G) $$
where:
- $ G $ is a graph.
- $ H(G) $ is the set of substructures in $ G $.

In [6]:
!pip install karateclub networkx
from karateclub import Graph2Vec
import networkx as nx

# Create example graphs
G1 = nx.erdos_renyi_graph(20, 0.2)
G2 = nx.barabasi_albert_graph(20, 2)
G3 = nx.watts_strogatz_graph(20, 4, 0.1)
graphs = [G1, G2, G3]

# Train Graph2Vec
graph2vec = Graph2Vec(dimensions=128, wl_iterations=2, min_count=1)
graph2vec.fit(graphs)

# Get graph embeddings
graph_embeddings = graph2vec.get_embedding()
print(graph_embeddings.shape)  # (num_graphs, embedding_dim)

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 99.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 127.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 90.2 MB/s eta 0:00:00:00:01
  Created wheel for karateclub: filename=karateclub-1.3.3-py3-none-any.whl size=101987 sha256=c482faf274e620e972692151c75b6e930737d0e2a4c2b2cdebed4a3ae1aad6a6
  Stored in di

## 6. DiffPool (Hierarchical Graph Pooling)
DiffPool is a hierarchical graph embedding approach where node embeddings are aggregated in a differentiable way to create coarser representations of the graph.

### **DiffPool Forward Pass:**
$$ Z = \text{softmax}(S) X $$
- $ S $ is the learned assignment matrix.
- $ X $ is the node feature matrix.
- $ Z $ is the pooled representation.

In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import DenseGCNConv, dense_diff_pool

class DiffPoolNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, num_clusters):
        super(DiffPoolNet, self).__init__()
        self.gcn1 = DenseGCNConv(in_channels, hidden_channels)
        self.gcn2 = DenseGCNConv(hidden_channels, num_clusters)

    def forward(self, x, adj):
        x = self.gcn1(x, adj).relu()
        s = self.gcn2(x, adj)
        x, adj, link_loss, ent_loss = dense_diff_pool(x, adj, s)  # Unpack all values
        return x, adj, link_loss, ent_loss  # Return all values

# Example usage
num_nodes, num_features, num_clusters = 10, 5, 3
x = torch.rand((num_nodes, num_features))
adj = torch.rand((num_nodes, num_nodes))

model = DiffPoolNet(num_features, 16, num_clusters)
pooled_x, pooled_adj, link_loss, ent_loss = model(x, adj)  # Correct unpacking

print(f"Pooled X shape: {pooled_x.shape}")
print(f"Pooled Adjacency Matrix shape: {pooled_adj.shape}")
print(f"Link Loss: {link_loss.item()}, Entropy Loss: {ent_loss.item()}")


Pooled X shape: torch.Size([1, 3, 16])
Pooled Adjacency Matrix shape: torch.Size([1, 3, 3])
Link Loss: 0.031235093250870705, Entropy Loss: 1.0217900276184082


### **Summary of Graph Embedding Methods**
- **GCN + Pooling**: Uses Graph Convolutional Networks with global mean pooling for whole-graph embedding.
- **Graph2Vec**: Learns graph embeddings using Weisfeiler-Lehman subtree features and Word2Vec.
- **DiffPool**: A hierarchical pooling method for learning coarser graph representations.